Observational Causal Inference:
===

IP weighting, stratification, and doubly robust estimators

Links:
 - Book: https://www.hsph.harvard.edu/miguel-hernan/causal-inference-book/
 - Code (Python, statsmodels): https://github.com/jrfiedler/causal_inference_python_code



In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.dpi'] = 100
#matplotlib.rcParams['font.family'] = "serif"

In [ ]:
from tqdm import tqdm

In [ ]:
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
def get_synthetic_data(
    n_total=1800,
    n_control=300,
    n_treatment=100,
):
    """
    Hypothetical population of users who will enroll in a study.
    
    The outcome is a continuous count: y_post
    Variables:
     - y_pre : Previous level of the primary outcome pre-treatment
     - cont1: Continuous variable
     - b1: Binary variable
     - cat1: Three-level categorical variable
     - I: an unobserved variable representing *interest in enrolling*
     - E: 1 if the user *chose to enroll* else 0
     - T: 1 if the user was treated else 0
    """
    n_enrolled = n_control + n_treatment
    
    I = np.random.beta(0.5, 2, size=n_total)
    
    # identify enrolled users
    p = I + np.random.beta(0.5, 2, size=n_total) / 5
    p = p / p.sum()
    E_inds = np.random.choice(np.arange(n_total), size=n_enrolled, replace=False, p=p)
    E = np.zeros(n_total)
    E[E_inds] = 1
    
    T_inds = np.random.choice(E_inds, size=n_treatment, replace=False)
    Tr = np.zeros(n_total)
    Tr[T_inds] = 1
    assert Tr.sum() == n_treatment
    
    #y_pre = np.random.poisson(lam=0.4, size=n_total)
    #lam_post = ((y_pre + 0.1) / (y_pre.max() + 0.1)) * 0.4
    #y_post = np.random.poisson(lam=lam_post, size=len(y_pre))
    
    y_pre = np.random.exponential(1, size=n_total) + np.random.normal(loc=I, scale=0.5, size=n_total)
    y_pre = np.maximum(y_pre, 0).round()
    
    y_post = y_pre * 0.5
    y_post += np.random.normal(loc=y_post, scale=0.5, size=n_total)
    
    #y_post = y_pre * 0.9 + 2 * np.random.normal(loc=-0.5, size=n_total)
    true_effect = 1
    y_post[Tr == 1] += np.random.normal(loc=true_effect, scale=0.1, size=n_treatment)
    #interest_effect = 1
    #y_post += 10 * np.random.normal(loc=I, scale=0.4, size=n_total)
    y_post = np.maximum(y_post, 0).round()
    
    
    df = pd.DataFrame.from_dict({
        'E': E,
        'I': I,
        'p_enroll': p,
        'y_pre': y_pre,
        'y_post': y_post,
        'Tr': Tr,
    })
    
    return df

df = get_synthetic_data()

In [ ]:
def get_synthetic_data(
    n_total=1800,
    n_control=300,
    n_treatment=100,
):
    n_enrolled = n_control + n_treatment
    
    I = np.random.normal(size=n_total)
    
    # identify enrolled users
    p = I + np.random.normal(loc=0, scale=0.1, size=n_total)
    p = (p - p.min())
    p = p / p.sum()
    E_inds = np.random.choice(np.arange(n_total), size=n_enrolled, replace=False, p=p)
    E = np.zeros(n_total)
    E[E_inds] = 1
    
    # select treatment and control groups
    T_inds = np.random.choice(E_inds, size=n_treatment, replace=False)
    Tr = np.zeros(n_total)
    Tr[T_inds] = 1
    assert Tr.sum() == n_treatment
        
    y_pre = 2 * I + np.random.normal(loc=0, scale=2, size=n_total)
    #y_pre = np.maximum(y_pre, 0).round()
    
    y_post = y_pre + np.random.normal(loc=-1, scale=0.5, size=n_total)
    y_post += 0.1*I  # residual effect of interest on y_post
    
    true_effect = 2
    y_post[Tr == 1] += np.random.normal(loc=true_effect, scale=0.01, size=n_treatment)
    #y_post = np.maximum(y_post, 0).round()
    
    
    df = pd.DataFrame.from_dict({
        'E': E,
        'I': I,
        'p_enroll': p,
        'y_pre': y_pre,
        'y_post': y_post,
        'Tr': Tr,
    })
    
    return df

df = get_synthetic_data()

In [ ]:
df.sample(n=2)

In [ ]:
df.agg(np.ptp)

In [ ]:
true_effects = []
naive_effects = []
for i in tqdm(range(1000)):
    df = get_synthetic_data()
    true_effect = df[(df.E == 1)&(df.Tr == 1)].y_post.mean() - df[(df.E == 1)&(df.Tr == 0)].y_post.mean()
    naive_effect = df[df.Tr == 1].y_post.mean() - df[df.Tr == 0].y_post.mean()
    true_effects.append(true_effect)
    naive_effects.append(naive_effect)
true_effects = np.array(true_effects)
naive_effects = np.array(naive_effects)
    
fig, axes = plt.subplots(1, 2)

ax = axes[0]
ax.hist(true_effects)
ax.set_title("True effect")

ax = axes[1]
ax.hist(naive_effects - true_effects)
ax.set_title("Naive effect - True effect")

plt.show()
true_effects.mean(), naive_effects.mean(), (naive_effects - true_effects).mean()

In [ ]:
# this is the true effect, as discovered via randomized experiment
# and the naive effect, not controlling for probability of enrollment
df[(df.E == 1)&(df.Tr == 1)].y_post.mean() - df[(df.E == 1)&(df.Tr == 0)].y_post.mean(),\
df[df.Tr == 1].y_post.mean() - df[df.Tr == 0].y_post.mean()

In [ ]:
df.corr()

In [ ]:
cols = df.columns
ncols = 3
nrows = int(np.ceil(len(cols) / ncols))
fig, axes = plt.subplots(nrows, ncols, figsize=(8, nrows*4))
for ax, col in zip(axes.flatten(), cols):
    nbins = min(df[col].nunique(), 100)
    hist, _ = np.histogram(df[col], bins=nbins)
    log = np.ptp(hist) > 1000  # three orders of magnitude...
    ax.hist(df[col], bins=nbins, log=log)
    ax.set_xlabel(col)
plt.show()

In [ ]:
cols = df.columns
ncols = 3
nrows = int(np.ceil(len(cols) / ncols))
fig, axes = plt.subplots(nrows, ncols, figsize=(8, nrows*4))
for ax, col in zip(axes.flatten(), cols):
    nbins = min(df[col].nunique(), 30)
    hist, _ = np.histogram(df[col], bins=nbins)
    log = np.ptp(hist) > 1000  # three orders of magnitude...
    ax.hist([df[df.Tr == 1][col], df[df.Tr == 0][col]], bins=nbins, log=log, density=True)
    ax.set_xlabel(col)
plt.show()

In [ ]:
for name, group in df.groupby('Tr'):
    print(f"Tr=={name} {group.y_pre.mean():.3f} {(group.y_post - group.y_pre).mean():.3f}")

In [ ]:
df[(df.E == 1)&(df.Tr == 1)].y_post.mean() - df[(df.E == 1)&(df.Tr == 0)].y_post.mean(),\
df[df.Tr == 1].y_post.mean() - df[df.Tr == 0].y_post.mean()

In [ ]:
md = smf.ols(formula='y_post ~ y_pre + Tr', data=df)
res = md.fit()
res.summary()

In [ ]:
# if we got to observe I, we get closer to the true effect with an OLS model
md = smf.ols(formula='y_post ~ y_pre + Tr + I', data=df)
res = md.fit()
res.summary()

In [ ]:
# we can also get close by including only enrolled authors
# (still upwardly biased though)
md = smf.ols(formula='y_post ~ y_pre + Tr', data=df[df.E == 1])
res = md.fit()
res.summary()

# IP Weighting

https://github.com/jrfiedler/causal_inference_python_code/blob/master/chapter12.ipynb

In [ ]:
def logit_ip_f(df, use_I=False):
    """
    Create the f(y|X) part of IP weights using logistic regression
    
    Adapted from https://github.com/jrfiedler/causal_inference_python_code/blob/master/chapter12.ipynb
    
    Parameters
    ----------
    df : Pandas DataFrame
    
    Returns
    -------
    Numpy array of IP weights
    
    """
    formula = 'Tr ~ y_pre'
    if use_I:
        formula = 'Tr ~ y_pre + I'
    model = smf.logit(formula=formula, data=df)
    res = model.fit(disp=0)
    #print(res.summary().tables[1])
    weights = np.zeros(len(df))
    weights[df.Tr == 1] = res.predict(df[df.Tr == 1])
    weights[df.Tr == 0] = (1 - res.predict(df[df.Tr == 0]))
    return weights

In [ ]:
weights = logit_ip_f(df)
weights = 1 / weights
plt.hist(weights, bins=50, log=True)
plt.show()

In [ ]:
print('IP weights')
print('   min: {:>5.2f}   expected:  X'.format(weights.min()))
print('   max: {:>5.2f}   expected: Y'.format(weights.max()))
print('  mean: {:>5.2f}   expected:  Z'.format(weights.mean()))

In [ ]:
wls = smf.wls(formula='y_post ~ Tr', data=df, weights=weights) 
res = wls.fit()
res.summary().tables[1]

In [ ]:
est = res.params.Tr
conf_ints = res.conf_int(alpha=0.05, cols=None)
lo, hi = conf_ints[0]['Tr'], conf_ints[1]['Tr']

print('           estimate   95% C.I.')
print(f'theta_1     {est:>6.2f}   ({lo:>0.1f}, {hi:>0.1f})')

In [ ]:
# notably, this estimate appears not to be any better than the naive estimate
df[df.Tr == 1].y_post.mean() - df[df.Tr == 0].y_post.mean()

# Standardization

Chapter 13

Code: https://github.com/jrfiedler/causal_inference_python_code/blob/master/chapter13.ipynb

In [ ]:
md = smf.ols(formula='y_post ~ y_pre + Tr', data=df)
res = md.fit()
res.summary().tables[1]

In [ ]:
block2 = df.copy()
block2.Tr = 0
block2_pred = res.predict(block2)

block3 = df.copy()
block3.Tr = 1
block3_pred = res.predict(block3)

In [ ]:
orig_mean = res.predict(df).mean()
block2_mean = block2_pred.mean()
block3_mean = block3_pred.mean()
est_diff = block3_mean - block2_mean

print('original mean prediction: {:>0.2f}'.format(orig_mean))
print()
print(' block 2 mean prediction: {:>0.2f}'.format(block2_mean))
print(' block 3 mean prediction: {:>0.2f}'.format(block3_mean))
print()
print('  causal effect estimate: {:>0.2f}'.format(est_diff))

How to report this?

The standardized mean for our participants was X, while the standardized mean in the pseudo-control was Y; thus, our estimate of the mean causal impact of the recommender intervention on journaling is X-Y additional journals.

"To obtain a doubly robust estimate of the average causal effect, first estimate the IP weight   = 1 (|)
as described in the previous chapter. Then fit an outcome regression model like the one described in this chapter–a
generalized linear model with a canonical link–for E[ | =   =  ] that adds the covariate , where  =   if
 = 1 and  = −  if  = 0. Finally, use the predicted values from the outcome model to obtain the standardized
mean outcomes under  = 1 and  = 0. The difference of the standardized mean outcomes is now doubly robust.
That is, under exchangeability and positivity given , this estimator consistently estimates the average causal effect if
either the model for the treatment or the model for the outcome is correct, without knowing which of the two models
is the correct one."

In [ ]:
# Pg. 167, "A doubly robust estimator"
# This is the Bang and Robins (2005) doubly robust estimator 
# "for the average causal effect of a dichotomous treatment on an outcome"

# "first estimate the IP weight"
weights = logit_ip_f(df)
weights = 1 / weights

# "then fit an outcome regression model ... that adds the covariate R"
# R is W if Tr == 1 else -W
block1 = df.copy()
block1['R'] = weights
block1.loc[block1.Tr == 0, 'R'] *= -1
md = smf.ols(formula='y_post ~ y_pre + Tr + R', data=block1)
res = md.fit()
print(res.summary().tables[1])

# "Finally, use the predicted values from the outcome model to obtain the standardized mean outcomes"
block2 = block1.copy()
block2.Tr = 0
#block2.W = 0  # unsure if we're supposed to maintain the weights... but it doesn't seem to matter
block3 = block1.copy()
block3.Tr = 1
#block3.W = 0
block2_pred = res.predict(block2)
block3_pred = res.predict(block3)
block3_pred.mean() - block2_pred.mean()

# Simulations

In [ ]:
results = []
for i in tqdm(range(1000)):
    df = get_synthetic_data()
    
    # blocks needed for standardization
    block2 = df.copy()
    block2.Tr = 0
    block3 = df.copy()
    block3.Tr = 1
    
    # basic regression estimates
    # that "adjust for" confounders
    # plus standardization
    md = smf.ols(formula='y_post ~ y_pre + Tr', data=df)
    res = md.fit()
    modeled_observational_effect = res.params.Tr
    block2_pred = res.predict(block2)
    block3_pred = res.predict(block3)
    standardized_effect = block3_pred.mean() - block2_pred.mean()
    
    # ... and with an interaction effect
    md = smf.ols(formula='y_post ~ y_pre + Tr + y_pre*Tr', data=df)
    res = md.fit()
    modeled_observational_effect_int = res.params.Tr
    block2_pred = res.predict(block2)
    block3_pred = res.predict(block3)
    standardized_effect_int = block3_pred.mean() - block2_pred.mean()
    
    # ... and with the I covariate
    md = smf.ols(formula='y_post ~ y_pre + Tr + I', data=df)
    res = md.fit()
    modeled_observational_effect_I = res.params.Tr
    #print(res.summary().tables[1])
    block2_pred = res.predict(block2)
    block3_pred = res.predict(block3)
    standardized_effect_I = block3_pred.mean() - block2_pred.mean()
    
    # IP weighting and the Bang-Robins doubly robust (DR) estimator
    weights = logit_ip_f(df)
    weights = 1 / weights
    wls = smf.wls(formula='y_post ~ Tr', data=df, weights=weights) 
    res = wls.fit(disp=0)
    ip_weighted_effect = res.params.Tr
    
    block1 = df.copy()
    block1['R'] = weights
    block1.loc[block1.Tr == 0, 'R'] *= -1
    md = smf.ols(formula='y_post ~ y_pre + Tr + R', data=block1)
    res = md.fit()
    block2 = block1.copy()
    block2.Tr = 0
    block3 = block1.copy()
    block3.Tr = 1
    block2_pred = res.predict(block2)
    block3_pred = res.predict(block3)
    dr_effect = block3_pred.mean() - block2_pred.mean()
    
    # ... and with the I covariate
    weights = logit_ip_f(df, use_I=True)
    weights = 1 / weights
    wls = smf.wls(formula='y_post ~ Tr', data=df, weights=weights) 
    res = wls.fit(disp=0)
    ip_weighted_effect_I = res.params.Tr
    
    block1 = df.copy()
    block1['R'] = weights
    block1.loc[block1.Tr == 0, 'R'] *= -1
    md = smf.ols(formula='y_post ~ y_pre + Tr + I + R', data=block1)
    res = md.fit()
    block2 = block1.copy()
    block2.Tr = 0
    block3 = block1.copy()
    block3.Tr = 1
    block2_pred = res.predict(block2)
    block3_pred = res.predict(block3)
    dr_effect_I = block3_pred.mean() - block2_pred.mean()
    
    # stabilized IP weighting
    weights = logit_ip_f(df)
    weights = 1 / weights
    pct_treated = df.Tr.mean()
    weights[df.Tr == 1] = pct_treated * weights[df.Tr == 1]
    weights[df.Tr == 0] = (1 - pct_treated) * weights[df.Tr == 0]
    wls = smf.wls(formula='y_post ~ Tr', data=df, weights=weights) 
    res = wls.fit(disp=0)
    sip_weighted_effect = res.params.Tr
    
    # ... and with the I covariate
    weights = logit_ip_f(df, use_I=True)
    weights = 1 / weights
    pct_treated = df.Tr.mean()
    weights[df.Tr == 1] = pct_treated * weights[df.Tr == 1]
    weights[df.Tr == 0] = (1 - pct_treated) * weights[df.Tr == 0]
    wls = smf.wls(formula='y_post ~ Tr', data=df, weights=weights) 
    res = wls.fit(disp=0)
    sip_weighted_effect_I = res.params.Tr
    
    results.append({
        'experimental_effect': df[(df.E == 1)&(df.Tr == 1)].y_post.mean() - df[(df.E == 1)&(df.Tr == 0)].y_post.mean(),
        'naive_observational_effect': df[df.Tr == 1].y_post.mean() - df[df.Tr == 0].y_post.mean(),
        'modeled_observational_effect': modeled_observational_effect,
        'ip_weighted_effect': ip_weighted_effect,
        'sip_weighted_effect': sip_weighted_effect,
        'standardized_effect': standardized_effect,
        'dr_effect': dr_effect,
        'modeled_observational_effect_I': modeled_observational_effect_I,
        'ip_weighted_effect_I': ip_weighted_effect_I,
        'sip_weighted_effect_I': sip_weighted_effect_I,
        'standardized_effect_I': standardized_effect_I,
        'dr_effect_I': dr_effect_I,
        'modeled_observational_effect_int': modeled_observational_effect_int,
        'standardized_effect_int': standardized_effect_int,
    })
rdf = pd.DataFrame(results)
rdf.sample(n=2)

In [ ]:
rdf.agg(['mean', 'std']).T

In [ ]:
"""
Key observation: confounders need to be specified correctly! if they're not, then modeling won't help

A few notes from simulations:

If I -> Y_pre and Y_pre -> Y_post but NOT I -> Y_post, then we seem to be mostly fine?
If I -> Y_pre and Y_pre -> Y_post but ALSO I -> Y_post, then we seem to be screwed... in fact, IP weighting just makes things worse

The standardized effects are extremely similar to the modeled observational effect.

The Doubly Robust estimates are, shockingly, actually much better, which is nice.

"""
for col in [
    'naive_observational_effect', 
    'modeled_observational_effect', 
    'ip_weighted_effect', 
    'sip_weighted_effect', 
    'standardized_effect',
    'dr_effect',
    'modeled_observational_effect_I', 
    'ip_weighted_effect_I', 
    'sip_weighted_effect_I',
    'standardized_effect_I',
    'dr_effect_I',
    'modeled_observational_effect_int',
    'standardized_effect_int',
]:
    diff = rdf[col] - rdf.experimental_effect
    print(f"{col:>35} diff    {diff.mean():.4f}")